In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import shutil

In [2]:
os.getcwd()

'/opt/zhuoyao_workspace/github/final_medical_cervix/src/CLASSIFY/inceptionV3_final_1/datasets'

In [3]:
os.listdir(os.getcwd())

['classify_denoise_2018',
 'classify_nodenoise_2019',
 'classify_nodenoise_2018',
 'classify_bak',
 '.ipynb_checkpoints',
 'classify_darken_2018',
 'classify_denoise_2019',
 'classify',
 'Tuning_Dataset.ipynb',
 'classify_recrop_2018',
 'test.txt',
 'classify_recrop_2019',
 '2018-12-14',
 '20190205_data',
 '2019_0205_dst',
 'train_labels_nodenoise_2018.csv_bak',
 'train_labels_nodenoise_2019.csv_bak',
 'train_labels_nodenoise_2019.csv',
 'train_labels_nodenoise_2018.csv',
 'classify_test',
 'train_labels_test.csv',
 'train_labels_test_dummy.csv',
 'classify_recrop',
 'clean.sh']

In [4]:
def crop(img, index, box, dst_path):
    x1, y1, w1, h1 = box
    cropped = img[y1:y1 + h1, x1:x1 + w1, :]
    cropped_filename = os.path.join(dst_path, str(index) + '.png')
    if not cv2.imwrite(cropped_filename, cropped):
        print("Image copy error, %s" % cropped_filename)
    
def draw_box(img, index, box):
    x1, y1, w1, h1 = box
    
    cv2.rectangle(img, (x1, y1), (x1 + w1, y1 + h1), (0, 255, 0), 1)
    #cv2.drawContours(img,cnt,-1,(0,255,0),1)
    cv2.putText(img, str(index), (x1, y1 + h1), cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (0, 255, 255), lineType=cv2.LINE_AA) 

# Copy missing images

l = ['NILM_IMG001x005.png_output',
'NILM_IMG001x010.png_output',
'NILM_IMG002x006.png_output',
'NILM_IMG002x014.png_output',
'NILM_IMG002x017.png_output',
'NILM_IMG003x002.png_output',
'NILM_IMG003x015.png_output',
'NILM_IMG004x023.png_output',
'NILM_IMG038x024.png_output',
'NILM_IMG038x025.png_output',
'NILM_IMG039x004.png_output',
'NILM_IMG039x005.png_output',
'NILM_IMG039x023.png_output',
'NILM_IMG039x024.png_output',
'NILM_IMG039x025.png_output']

src_dir = '/opt/zhuoyao_workspace/Datasets/private_cervical/origin/TP18JAN-08（NILM）/Images/'
dst_dir = '/opt/zhuoyao_workspace/final_medical_cervix_2/datasets/origin_train/origin_10_1_train/'
for file in l:
    file_name1 = file.split('_')[1].split('.')[0]
    filename = file_name1+'.JPG'
    src = src_dir + filename
    dst = dst_dir + 'NILM_'+ file_name1 + '.png'
    #print(dst)
    #print(src)
    
    img = cv2.imread(src)
    cv2.imwrite(dst, img)
    

# Original Dataset

print(os.popen('rm -rf ./classify').read())
print(os.popen('cp -rf ./classify_nodenoise_2019 ./classify').read())

# Denoising



#origin_dirs = ['/opt/zhuoyao_workspace/github/final_medical_cervix/datasets/origin_train/origin_12_06',
             '/opt/zhuoyao_workspace/github/final_medical_cervix/datasets/origin_train/origin_10_1_train']
#origin_dirs = ['/opt/zhuoyao_workspace/github/final_medical_cervix/datasets/origin_train/origin_10_1_train']
origin_dirs = ['/opt/zhuoyao_workspace/github/final_medical_cervix/datasets/origin_train/origin_01_15']
src_dir = './classify_nodenoise_2019'
dst_dir = './classify_denoise_2019'


for origin_dir in origin_dirs:
    print(origin_dir)
    files = os.listdir(origin_dir)
    print(files)


    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    for file in files:
        img_path = os.path.join(origin_dir, file)
        img = cv2.imread(img_path)
        #denoising
        img = cv2.fastNlMeansDenoisingColored(img, None, 6, 6, 7, 21)
    
        #prepare dir
        folder_l1 = os.path.join(dst_dir, file + '_output')
        folder1_l2 = os.path.join(folder_l1, 'preview')
        folder2_l2 = os.path.join(folder_l1, 'crops')
        if not os.path.exists(folder1_l2):
            os.makedirs(folder1_l2)
        if not os.path.exists(folder2_l2):
            os.makedirs(folder2_l2)
        preview_path = os.path.join(folder1_l2, 'preview.png')
        ind_path = os.path.join(folder1_l2, 'index_file.txt')
    
        #crop files
        co_file = os.path.join(src_dir, file + '_output', 'preview/index_file.txt')
        co_df = pd.read_csv(co_file, sep=' ', names=['x1','y1','w1','h1'])
        for index, row in co_df.iterrows():
            box = row.values
            crop(img, index, box, folder2_l2)
            
        for index, row in co_df.iterrows():
            box = row.values
            draw_box(img, index, box)
    
        #save image    
        cv2.imwrite(preview_path, img)
        #copy index_file
        shutil.copy(co_file, ind_path)

print(os.popen('rm -rf ./classify').read())
print(os.popen('cp -rf {} ./classify'.format(dst_dir)).read())

# Darken

First choose files to generate dark dataset, then copy the original(nodenoise) to classify and replace the files with dark dataset

origin_dirs = ['/opt/zhuoyao_workspace/final_medical_cervix_2/datasets/origin_train/origin_10_1_train',
              '/opt/zhuoyao_workspace/final_medical_cervix_2/datasets/origin_train/origin_12_06']
src_dir = './classify_nodenoise_2018'
dst_dir = './classify_darken_2018'
darken_ratio = 0.7
select_percent = 0.3


print(os.popen('rm -rf ./classify_darken_2018').read())

np.random.seed(10)
for origin_dir in origin_dirs:
    print(origin_dir)
    files = os.listdir(origin_dir)
    files = np.random.choice(files, int(len(files)*select_percent), replace=False)
    print(files)


    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    for file in files:
        img_path = os.path.join(origin_dir, file)
        img = cv2.imread(img_path)
        img = (img * darken_ratio).astype(np.uint8)
    
        #prepare dir
        folder_l1 = os.path.join(dst_dir, file + '_output')
        folder1_l2 = os.path.join(folder_l1, 'preview')
        folder2_l2 = os.path.join(folder_l1, 'crops')
        if not os.path.exists(folder1_l2):
            os.makedirs(folder1_l2)
        if not os.path.exists(folder2_l2):
            os.makedirs(folder2_l2)
        preview_path = os.path.join(folder1_l2, 'preview.png')
        ind_path = os.path.join(folder1_l2, 'index_file.txt')
    
        #crop files
        co_file = os.path.join(src_dir, file + '_output', 'preview/index_file.txt')
        co_df = pd.read_csv(co_file, sep=' ', names=['x1','y1','w1','h1'])
        for index, row in co_df.iterrows():
            box = row.values
            crop(img, index, box, folder2_l2)
            draw_box(img, index, box)
    
        #save image    
        cv2.imwrite(preview_path, img)
        #copy index_file
        shutil.copy(co_file, ind_path)
        
print(os.popen('rm -rf ./classify').read())
print(os.popen('cp -rf {} ./classify'.format(src_dir)).read())
print(os.popen('cp -rf {}/* ./classify'.format(dst_dir)).read())

# Recropping

In [5]:
def resize_box(df):
    def square_x(serie):
        x = serie['x1'] + int(serie['w1']/2) - int(square_edges/2)
        return max(x, 0)
    def square_y(serie):
        y = serie['y1'] + int(serie['h1']/2) - int(square_edges/2)
        return max(y, 0)
        
    if len(co_df)!=0:    
        df['x1'] = df.apply(square_x, axis=1)
        df['y1'] = df.apply(square_y, axis=1)
        df['w1'] = square_edges
        df['h1'] = square_edges
    return df

In [6]:
#origin_dirs = ['/opt/zhuoyao_workspace/github/final_medical_cervix/datasets/origin_train/origin_10_1_train',
#              '/opt/zhuoyao_workspace/github/final_medical_cervix/datasets/origin_train/origin_12_06']
#src_dir = './classify_nodenoise_2018'
#dst_dir = './classify_recrop_2018'
origin_dirs = ['/opt/zhuoyao_workspace/github/final_medical_cervix/datasets/origin_train/origin_01_15']
src_dir = './classify_nodenoise_2019'
dst_dir = './classify_recrop_2019'
square_edges = 75


for origin_dir in origin_dirs:
    print(origin_dir)
    files = os.listdir(origin_dir)
    print(files)


    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)

    for file in files:
        img_path = os.path.join(origin_dir, file)
        img = cv2.imread(img_path)
    
        #prepare dir
        folder_l1 = os.path.join(dst_dir, file + '_output')
        folder1_l2 = os.path.join(folder_l1, 'preview')
        folder2_l2 = os.path.join(folder_l1, 'crops')
        if not os.path.exists(folder1_l2):
            os.makedirs(folder1_l2)
        if not os.path.exists(folder2_l2):
            os.makedirs(folder2_l2)
        preview_path = os.path.join(folder1_l2, 'preview.png')
        ind_path = os.path.join(folder1_l2, 'index_file.txt')
        
        #crop files
        co_file = os.path.join(src_dir, file + '_output', 'preview/index_file.txt')
        co_df = pd.read_csv(co_file, sep=' ', names=['x1','y1','w1','h1'])
        
        #original crops
        origin_crops_path = os.path.join(src_dir, file + '_output', 'crops') 
        #print(origin_crops_path)
        origin_crops = os.listdir(origin_crops_path)
        #print(origin_crops)
        if len(origin_crops) != len(co_df):
            print("File:{}, len_df: {}, num_origin_crops: {}".format(file, 
                                                                   len(co_df),
                                                                   len(origin_crops)))
            #print(co_df.index)
            for crops in origin_crops:
                if int(crops.split('.')[0]) not in co_df.index:
                    src = os.path.join(origin_crops_path, crops)
                    dst = os.path.join(*origin_crops_path.split('/')[:-1])
                    #print(dst)
                    shutil.move(src, dst)
                    #print(co_df.index)
                                                            
        co_df = resize_box(co_df)

        
        for index, row in co_df.iterrows():
            box = row.values
            crop(img, index, box, folder2_l2)
            
        for index, row in co_df.iterrows():
            box = row.values
            draw_box(img, index, box)
    
        #save image    
        cv2.imwrite(preview_path, img)
        #copy index_file
        co_df.to_csv(ind_path, sep=' ', header=False)

/opt/zhuoyao_workspace/github/final_medical_cervix/datasets/origin_train/origin_01_15
['HSIL_IMG042x013.png', 'HSIL_IMG042x007.png', 'HSIL_IMG039x010.png', 'HSIL_IMG039x011.png', 'HSIL_IMG042x006.png', 'HSIL_IMG042x012.png', 'HSIL_IMG042x004.png', 'HSIL_IMG042x010.png', 'HSIL_IMG039x013.png', 'HSIL_IMG039x012.png', 'HSIL_IMG042x011.png', 'HSIL_IMG042x005.png', 'HSIL_IMG042x001.png', 'HSIL_IMG042x015.png', 'HSIL_IMG039x016.png', 'HSIL_IMG039x017.png', 'HSIL_IMG042x014.png', 'HSIL_IMG042x002.png', 'HSIL_IMG039x015.png', 'HSIL_IMG039x014.png', 'HSIL_IMG042x003.png', 'HSIL_IMG042x017.png', 'HSIL_IMG040x015.png', 'HSIL_IMG040x001.png', 'HSIL_IMG041x004.png', 'HSIL_IMG041x010.png', 'HSIL_IMG041x011.png', 'HSIL_IMG041x005.png', 'HSIL_IMG040x014.png', 'HSIL_IMG040x002.png', 'HSIL_IMG040x016.png', 'HSIL_IMG041x013.png', 'HSIL_IMG041x007.png', 'HSIL_IMG041x006.png', 'HSIL_IMG041x012.png', 'HSIL_IMG040x017.png', 'HSIL_IMG040x003.png', 'HSIL_IMG040x007.png', 'HSIL_IMG041x016.png', 'HSIL_IMG041x002

# Generate Train & Test

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
def copy_data_by_type(df,dst_dir):
    if not os.path.exists(dst_dir):
        os.mkdir(dst_dir)
    
    CLASS = ['Norm', 'LSIL', 'HSIL', 'HPV', 'SCC']
    gt = [0, 1, 2, 3, 5]

    for i, t in enumerate(gt):
        try:
            tmp_df = df.loc[df['gt']==t]
        except:
            tmp_df = df.loc[df['positive']==t]
        path = os.path.join(dst_dir, CLASS[i])
        if not os.path.exists(path):
            os.mkdir(path)
        for _, row in tmp_df.iterrows():
            src_path = row['name']
            names = src_path.split('/')[-3] + '_' + src_path.split('/')[-1]
            dst_path = os.path.join(path,names)
            shutil.copy(src_path, dst_path)

In [9]:
orgin_name = 'classify_nodenoise_2019'
dst_name = 'classify_recrop_2019'

if '2018' in orgin_name:
    df = pd.read_csv('./train_labels_nodenoise_2018.csv')
elif '2019' in orgin_name:
    df = pd.read_csv('./train_labels_nodenoise_2019.csv')
else:
    print("Not supported dataset.")

df['name'] = df.apply(lambda x: x['name'].replace(orgin_name, dst_name), axis=1)
df['name'] = df.apply(lambda x: '../' + x['name'], axis=1)

try: 
    y = df['gt']
except:
    y = df['positive']

train_set, test_set, _, _ =  train_test_split(df, y, test_size=0.2, 
                                              shuffle=True, stratify=y, random_state=10)

copy_data_by_type(train_set, './train_datasets')
copy_data_by_type(test_set, './test_datasets')